In [1]:
from GeneralTools.misc_fun import FLAGS

FLAGS.TENSORFLOW_VERSION


'1.13.0'

In [9]:
import tensorflow as tf
"""
Scope - continue

tf.variable_scope vs tf.name_scope
 -- variable_scope applies to constants, variables and operations
 -- name_scope applies to constants and operations, but not variables
"""

with tf.Graph().as_default():
    a1 = tf.constant(0.1, name='const_a1')
    b1 = tf.get_variable(
        name='var_b1', dtype=tf.float32, 
        initializer=a1)
    c1 = tf.multiply(a1, b1, name='op_1')

    with tf.variable_scope('layer01'):
        # variable scope applies to constants, variables and operations
        a2 = tf.constant(0.1, name='const_a2')
        b2 = tf.get_variable(
            name='var_b2', dtype=tf.float32, 
            initializer=a1)  # initializer could be a1, b1, a2
        c2 = tf.multiply(a2, b2, name='op_2')
        
    with tf.name_scope('layer02'):
        # name scope applies to constants and operations
        a3 = tf.constant(0.1, name='const_a3')
        # name scope does not apply to variables
        b3 = tf.get_variable(
            name='var_b3', dtype=tf.float32, 
            initializer=a1)  # initialize could be a1, b1, a2, b2, a3
        c3 = tf.multiply(a3, b3, name='op_3')
    
print("The name/operation of a1 is '{}'".format(a1.name))
print("The name/operation of b1 is '{}'".format(b1.name))
print("The name/operation of c1 is '{}'".format(c1.name))
print('------')
print("The name/operation of a2 is '{}'".format(a2.name))
print("The name/operation of b2 is '{}'".format(b2.name))
print("The name/operation of c2 is '{}'".format(c2.name))
print('------')
print("The name/operation of a3 is '{}'".format(a3.name))
print("The name/operation of b3 is '{}'".format(b3.name))
print("The name/operation of c3 is '{}'".format(c3.name))


The name/operation of a1 is 'const_a1:0'
The name/operation of b1 is 'var_b1:0'
The name/operation of c1 is 'op_1:0'
------
The name/operation of a2 is 'layer01/const_a2:0'
The name/operation of b2 is 'layer01/var_b2:0'
The name/operation of c2 is 'layer01/op_2:0'
------
The name/operation of a3 is 'layer02/const_a3:0'
The name/operation of b3 is 'var_b3:0'
The name/operation of c3 is 'layer02/op_3:0'


In [6]:
import tensorflow as tf
from GeneralTools.my_input import DatasetFromTensor

with tf.Graph().as_default():
    # load the data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    data_tr = DatasetFromTensor(
        (x_train, y_train), (tf.float32, tf.int32), 
        map_func=lambda x, y: (x/255.0, y),
        name='TrainData').schedule(10)
    xtr, ytr = data_tr.next()
    initializer = data_tr.initializer()
    
    with tf.Session() as sess:
        sess.run(initializer[0], feed_dict=initializer[1])
        xv = sess.run(xtr)
        print(xv.shape)


TrainData dataset output type is (tf.float32, tf.int32)
TrainData dataset output shape is (TensorShape([Dimension(28), Dimension(28)]), TensorShape([]))
Instructions for updating:
Colocations handled automatically by placer.


(10, 28, 28)


In [1]:
import tensorflow as tf
import os.path
from GeneralTools.misc_fun import FLAGS
FLAGS.DEFAULT_OUT = 'C:/Users/richa/PycharmProjects/ai_mechatronics/Results/test'
from GeneralTools.my_layer import SequentialNet
from GeneralTools.my_graph import MySession, graph_configure, prepare_folder

with tf.Graph().as_default():
    x = tf.random_normal((64, 28))
    y = tf.random_normal((64, ))
    # net_design = [{'name': 'l1', 'out': 33, 'op': 'c', 'in_reshape': (1, 28, 28)}, 
    #               {'name': 'l2', 'out': 66, 'op': 'c', 'stride': 2}, 
    #               {'name': 'l3', 'out': 11, 'op': 'c', 'stride': 2}]
    net_design = [{'name': 'l1', 'out': 33, 'op': 'd'}, 
                  {'name': 'l2', 'out': 66, 'op': 'd'}, 
                  {'name': 'l3', 'out': 1, 'op': 'd'}]
    mdl = SequentialNet(net_design, input_shape=[None, 28], name='MLP')
    
    y2 = mdl(x)
    
    loss = tf.losses.mean_squared_error(
        y, y2, weights=1.0, scope=None, 
        reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS)
    global_step, opt_op, _ = graph_configure(0.1)
    op = opt_op.minimize(loss, global_step=global_step)
    print(global_step)
    
    tf.summary.scalar('loss', loss)
    summary_op = tf.summary.merge_all()
    ckpt_folder, summary_folder, save_path = prepare_folder('test01')

    with MySession(do_save=True, load_ckpt=True) as sess:
        sess.full_run(
            op, loss, max_step=100, step_per_epoch=1000, 
            global_step=global_step, summary_op=summary_op, 
            summary_folder=summary_folder, ckpt_folder=ckpt_folder, 
            save_path=save_path, query_step=10)


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Adam Optimizer is used.


<tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Graph initialization finished...


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/test\test01_ckpt\test01.ckpt-100


Model reloaded from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/test\test01_ckpt\.


Epoch 0, global steps 109, query value ['<1.242>']
Epoch 0, global steps 119, query value ['<0.850>']
Epoch 0, global steps 129, query value ['<0.953>']


Epoch 0, global steps 139, query value ['<1.035>']
Epoch 0, global steps 149, query value ['<1.105>']
Epoch 0, global steps 159, query value ['<0.985>']
Epoch 0, global steps 169, query value ['<1.039>']
Epoch 0, global steps 179, query value ['<0.767>']
Epoch 0, global steps 189, query value ['<0.913>']
Epoch 0, global steps 199, query value ['<1.103>']


Training for 100 steps took 0.940 sec.
Session finished.


In [5]:
import tensorflow as tf
from my_classifier import MyClassifier

"""
Six-step workflow of data analysis:
1. prepare data
2. define the model
3. define the loss function
4. configure the optimizer
5. configure the summary
6. call a session
"""

# TODO: test this part and debug

graph = tf.Graph()
with graph.as_default():
    # load the data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # define the model
    net_design = [{'name': 'l1', 'out': 33, 'op': 'd', 'in_reshape': [784]}, 
                  {'name': 'l2', 'out': 66, 'op': 'd'}, 
                  {'name': 'l3', 'out': 1, 'op': 'd'}]
    mdl = MyClassifier(
        net_design, input_shape=[None, 28, 28], num_class=10, 
        optimizer='momentum', name='classifier')
    mdl.train(
        x_train, y_train, batch_size=60,
        map_func=lambda x, y: (x/255.0, y), 
        x_validate=x_test, y_validate=y_test, model_folder='trial01')
    mdl.test(x_test, y_test, map_func=lambda x, y: (x/255.0, y), model_folder='trial01')


1